In [ ]:
%pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp38-cp38-linux_x86_64.whl size=4438469 sha256=028a3d7c8cc3ebd35ac8f5c3a10747ce57b354889bf3c699ad598d72bbd72179
  Stored in directory: /root/.cache/pip/wheels/f3/35/48/0b9a7076995eea5ea64a7e4bc3f0f342f453080795276264e7
Successfully built mpi4py


In [ ]:
from mpi4py import MPI

comm = MPI. COMM_WORLD
rank = comm. Get_rank ()
numprocs = comm. Get_size ()

print (f'Hello from process {rank} of { numprocs }')

Hello from process 0 of 1


In [ ]:
from mpi4py import MPI
from numpy import empty , array , int32 , float64 , dot

# определяем коммуникатор comm – группу процессов, которые участвуют в вычислениях
comm = MPI. COMM_WORLD
# rank применяется для указания номера процесса
rank = comm. Get_rank ()


numprocs = comm. Get_size ()

if rank == 0:
  f1 = open('/content/in.dat', 'r')
  N = array(int32(f1. readline ()))
  M = array(int32(f1. readline ()))
  f1.close ()
else:
  N = array (0, dtype=int32)
  M = array (0, dtype=int32)

if rank == 0:
  comm.send (M, dest =1, tag =0)
else:
  N = comm.recv (source =0, tag =0)

print (f'Variable N on process {rank} is: {N}')

Exception: ignored

In [ ]:
from mpi4py import MPI
from numpy import empty , array , int32 , float64 , dot

comm = MPI. COMM_WORLD
rank = comm. Get_rank ()

numprocs = comm. Get_size ()

if rank == 0: 
    variable_to_share = 100      
else: 
    variable_to_share = None

# Отсылка значения всем, в т.ч. и себе
variable_to_share = comm.bcast(variable_to_share, root=0) 

print("process = %d" %rank + " variable shared  = %d " %variable_to_share)


process = 0 variable shared  = 100 


In [ ]:
import numpy
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.size
rank = comm.rank

array_size = 3

recvdata = numpy.zeros(array_size,dtype=int)
# Формируем массивы с исходными данными размером array_size
senddata = (rank+1)*numpy.arange(array_size,dtype=int)

print(" process %s sending %s " %(rank , senddata))

# Применяем групповое суммирование каждого элемента во всех процессах  
comm.Reduce(senddata,recvdata,root=0,op=MPI.SUM)

# и выводим в нулевом результат
if rank == 0: 
  print ('on task',rank,'after Reduce:    data = ',recvdata)

 process 0 sending [0 1 2] 
on task 0 after Reduce:    data =  [0 1 2]


In [ ]:
from mpi4py import MPI
from numpy import empty , array , int32 , float64 , dot

comm = MPI. COMM_WORLD
rank = comm. Get_rank ()

numprocs = comm. Get_size ()

if rank == 0: 
    array_to_share = [1]
else: 
    array_to_share = None


# Отсылка по элементу из массива всем процессам, в т.ч. и себе
recvbuf = comm.scatter(array_to_share, root=0)

print("process = %d" %rank + " recvbuf = %d " %recvbuf)

process = 0 recvbuf = 1 


In [ ]:
from mpi4py import MPI
from numpy import empty , array , int32 , float64 , dot

comm = MPI. COMM_WORLD
rank = comm. Get_rank ()
size = comm. Get_size ()


numprocs = comm. Get_size ()

if rank == 0: 
    array_share = [ 0 ]
    for i in range(1,size): 
        array_share.append( i )

else: 
    array_share = None


# Отсылка по элементу из массива всем процессам, в т.ч. и себе
recvbuf = comm.scatter(array_share, root=0)

# вычисляем по значению в каждом процессе
calcVal = recvbuf * 2

# собираем результат обратно в вектор
recv_arr = comm.gather(calcVal, root=0)

if rank == 5: 
  for i in range(0,size): 
       # выводим поэлементно полученный вектор
       print(" process %s receiving %s from process %s"  %(rank , recv_arr[i] , i))
